In [1]:
import os
from collections import OrderedDict
from torch.autograd import Variable
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
from util import html
import torch
import pickle

In [2]:
with open('test_opt.pi', 'rb') as f:
    opt = pickle.load(f)

In [3]:
opt.name = 'dressed2nude'
opt.label_nc = 0
opt.no_instance = True
opt.loadSize = 256
opt.batchSize = 1
opt.dataroot = './datasets/dressed2nude'

In [4]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)
# create website
web_dir = os.path.join(opt.results_dir, opt.name, '%s_%s' % (opt.phase, opt.which_epoch))
webpage = html.HTML(web_dir, 'Experiment = %s, Phase = %s, Epoch = %s' % (opt.name, opt.phase, opt.which_epoch))

CustomDatasetDataLoader
dataset [AlignedDataset] was created


In [5]:
if not opt.engine and not opt.onnx:
    model = create_model(opt)
    if opt.data_type == 16:
        model.half()
    elif opt.data_type == 8:
        model.type(torch.uint8)
            
    if opt.verbose:
        print(model)
else:
    from run_engine import run_trt_engine, run_onnx

GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14): InstanceNorm2d(1024, eps=1e-05, momentum=0.1, affine=False, track_runn

In [6]:
for i, data in enumerate(dataset):
    if i >= opt.how_many:
        break
    if opt.data_type == 16:
        data['label'] = data['label'].half()
        data['inst']  = data['inst'].half()
    elif opt.data_type == 8:
        data['label'] = data['label'].uint8()
        data['inst']  = data['inst'].uint8()
    if opt.export_onnx:
        print ("Exporting to ONNX: ", opt.export_onnx)
        assert opt.export_onnx.endswith("onnx"), "Export model file should end with .onnx"
        torch.onnx.export(model, [data['label'], data['inst']],
                          opt.export_onnx, verbose=True)
        exit(0)
    minibatch = 1 
    if opt.engine:
        generated = run_trt_engine(opt.engine, minibatch, [data['label'], data['inst']])
    elif opt.onnx:
        generated = run_onnx(opt.onnx, opt.data_type, minibatch, [data['label'], data['inst']])
    else:        
        generated = model.inference(data['label'], data['inst'])
        
    visuals = OrderedDict([('input_label', util.tensor2label(data['label'][0], opt.label_nc)),
                           ('synthesized_image', util.tensor2im(generated.data[0]))])
    img_path = data['path']
    print('process image... %s' % img_path)
    visualizer.save_images(webpage, visuals, img_path)

webpage.save()

/home/jean/bazar/pix2pixHD-master/models/pix2pixHD_model.py:130: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  input_label = Variable(input_label, volatile=infer)


process image... ['./datasets/dressed2nude/test_A/0.png']
process image... ['./datasets/dressed2nude/test_A/1.png']
process image... ['./datasets/dressed2nude/test_A/10.png']
process image... ['./datasets/dressed2nude/test_A/11.png']
process image... ['./datasets/dressed2nude/test_A/12.png']
process image... ['./datasets/dressed2nude/test_A/13.png']
process image... ['./datasets/dressed2nude/test_A/14.png']
process image... ['./datasets/dressed2nude/test_A/15.png']
process image... ['./datasets/dressed2nude/test_A/16.png']
process image... ['./datasets/dressed2nude/test_A/17.png']
process image... ['./datasets/dressed2nude/test_A/18.png']
process image... ['./datasets/dressed2nude/test_A/19.png']
process image... ['./datasets/dressed2nude/test_A/2.png']
process image... ['./datasets/dressed2nude/test_A/20.png']
process image... ['./datasets/dressed2nude/test_A/21.png']
process image... ['./datasets/dressed2nude/test_A/22.png']
process image... ['./datasets/dressed2nude/test_A/23.png']
